In [1]:
import os
import json
import numpy as np
import sqlite3
import matplotlib.pyplot as plt

In [2]:
### Configuration

SAVE_FIGURES = True
SQLITE_PATH = "/eos/user/m/mantydze/DQMIO2017/ZeroBias2017D.sqlite"

RUN = 302040
HISTOGRAM = "RPC/AllHits/SummaryHistograms/Occupancy_for_Barrel"

In [3]:
### Create export directory RUN/HISTOGRAM/

fig_name = HISTOGRAM.replace("/", ".")

# Run directory
if not os.path.exists(str(RUN)):
    os.makedirs(str(RUN))

# histogram directory
dir_name = os.path.join(str(RUN), fig_name)
if not os.path.exists(dir_name):
    os.makedirs(dir_name)

In [4]:
conn = sqlite3.connect(SQLITE_PATH)
conn.row_factory = sqlite3.Row
c = conn.cursor()

In [5]:
def create_heatmap(width, height, data, title="", filename=None):
    
    heatmap = np.array(data).reshape(height, width)
    
    # remove border
    heatmap = heatmap[1:height-1, 1:width-1]
    
    plt.figure(figsize=(15,10))
    plt.title(title)
    plt.colorbar(plt.pcolor(heatmap, cmap="rainbow"))
    
    # Annotate
    for y in range(heatmap.shape[0]):
        for x in range(heatmap.shape[1]):
            plt.text(x + 0.5, y + 0.5, '%d' % heatmap[y, x],
                    horizontalalignment='center',
                    verticalalignment='center',)
    
    if SAVE_FIGURES:
        plt.savefig(filename)    
        plt.close()

In [7]:
query = c.execute("Select * FROM monitorelements WHERE fromrun = ? and name = ?", (RUN, HISTOGRAM))

rows = query.fetchall()

row = dict(rows[0])
jvalue = json.loads((row["value"]))

width = jvalue["fXaxis"]["fNbins"] + 2 # +2 because border is added on both X and Y axis
height = jvalue["fYaxis"]["fNbins"] + 2
title = jvalue["fTitle"]
total = np.zeros(width*height) # Accumulated values during whole run

# Cross-check if numbers match
if jvalue["fNcells"] != (width*height):
    import sys
    sys.exit()

# Iterate over all lumisections
for row in rows:
    
    ls = row["fromlumi"]
    _filename = os.path.join(dir_name, "{fig_name}-{run}-{ls}.png".format(fig_name=fig_name, run=RUN, ls=ls))
    _title = "{title} LS #{ls}".format(title=title, ls=ls)

    # 2D histogram                                 
    if jvalue["_typename"] == "TH2F":
        create_heatmap(width, height, jvalue["fArray"], title=_title, filename=_filename)
        total = np.add(total, jvalue["fArray"])                     

# 2D histogram        
if jvalue["_typename"] == "TH2F":
    _filename = os.path.join(dir_name, "{fig_name}-{run}.png".format(fig_name=fig_name, run=RUN))
    create_heatmap(width, height, total, title=title, filename=_filename)